In [1]:
#working version is on pycharm on university computer

#on university computer it computes 5 overall topics (with speeches conflated)
# -other version computes 5 topics PER speech
#results are in the "summaries" directory 
#topics it gives are the areas of text to focus on (perhaps)

from __future__ import unicode_literals
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import os
import gensim
import pickle
import spacy
import nltk
import codecs

spacy.load('en')


def tokenize(text):
    parser = English()
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma


def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)


def prepare_text_for_lda(text):
    en_stop = set(nltk.corpus.stopwords.words('english'))
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


def main():
    nltk.download('wordnet')
    nltk.download('stopwords')
    text_data = []

    directory = '/homes/ahr18/PycharmProjects/topic_practice/Source/data'

    os.chdir(directory)

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            print(filename)
            f = open(filename)
            #lines = f.read()
            with codecs.open(filename, 'r', encoding='utf-8') as fg:
                for line in fg:
                    #prep the data
                    tokens = prepare_text_for_lda(line)
                    text_data.append(tokens)

    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]

    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('dictionary.gensim')

    #gets 5 topics for the entire set of speeches
    #in other version, this code is within the for loop and occurs for each
    #individual speech, giving 5 topics per speech (in summaries folder)
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
    ldamodel.save('model5.gensim')
    topics = ldamodel.print_topics(num_words=4)

    for topic in topics:
        print(topic)



if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'spacy'

In [3]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install spacy